In [1]:
import os

import fiona
from fiona.crs import to_string

In [2]:
path = '/home/madmex_user/results/'

In [3]:
shapefile = path + 'poligonos_agricultura.shp'

In [4]:
shapefile

'/home/madmex_user/results/poligonos_agricultura.shp'

In [5]:
with fiona.open(shapefile) as src:
    schema = src.schema
    fc = [x for x in src]
    crs = to_string(src.crs)

In [6]:
schema

{'properties': OrderedDict([('id', 'int:18'),
              ('clave_31', 'int:18'),
              ('descr_31', 'str:80'),
              ('clave_17', 'int:18'),
              ('descr_17', 'str:80'),
              ('area', 'float:24.15')]),
 'geometry': 'Polygon'}

In [7]:
fc[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 20),
              ('clave_31', 28),
              ('descr_31', 'Tierras Agrícolas'),
              ('clave_17', 14),
              ('descr_17', 'Tierras agrícolas'),
              ('area', 0.04)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-94.6764286178057, 17.997156068639345),
    (-94.67623985674925, 17.9971468373339),
    (-94.67624950426081, 17.996966217624106),
    (-94.67643826506946, 17.996975448914817),
    (-94.6764286178057, 17.997156068639345)]]}}

In [8]:
#[feat['geometry']['type'] for feat in fc]

In [9]:
schema['geometry'] = ('Polygon', 'MultiPolygon')

In [10]:
schema

{'properties': OrderedDict([('id', 'int:18'),
              ('clave_31', 'int:18'),
              ('descr_31', 'str:80'),
              ('clave_17', 'int:18'),
              ('descr_17', 'str:80'),
              ('area', 'float:24.15')]),
 'geometry': ('Polygon', 'MultiPolygon')}

In [11]:
filename =  os.path.expanduser('~') + '/' + 'poligonos_agricultura' + '.gpkg'

In [16]:
filename

'/home/madmex_user/poligonos_agricultura.gpkg'

In [17]:
with fiona.open(filename, 
               'w',
               layer='poligonos_agricultura',
               schema = schema,
               crs = crs,
               driver='GPKG') as dst:
    [dst.write(x) for x in fc]

#Successful write

## Create spatial index via ogr:

```
ogrinfo -SQL "select CreateSpatialIndex('poligonos_agricultura', 'geom')" poligonos_agricultura.gpkg
```

```
INFO: Open of `poligonos_agricultura.gpkg'
      using driver `GPKG' successful.
ERROR 1: Spatial index already existing

Layer name: SELECT
Geometry: Unknown (any)
Feature Count: 1
Layer SRS WKT:
(unknown)
CreateSpatialIndex: Integer (0.0)
OGRFeature(SELECT):0
  CreateSpatialIndex (Integer) = 0
```

## Check Rtree index

```
ogrinfo -SQL "SELECT id FROM rtree_poligonos_agricultura_geom r" poligonos_agricultura.gpkg |head -n 20
INFO: Open of `poligonos_agricultura.gpkg'
      using driver `GPKG' successful.

Layer name: SELECT
Geometry: None
Feature Count: 1203
Layer SRS WKT:
(unknown)
id: Integer (0.0)
OGRFeature(SELECT):0
  id (Integer) = 1075

OGRFeature(SELECT):1
  id (Integer) = 1067

OGRFeature(SELECT):2
  id (Integer) = 1022

OGRFeature(SELECT):3
  id (Integer) = 985
```

# Can't write to LUSTRE: (`/home/madmex_user/results/` is mapped to directory in `LUSTRE`)

In [40]:
filename =  os.path.expanduser('~') + '/results/' + 'poligonos_agricultura' + '.gpkg'

In [41]:
filename

'/home/madmex_user/results/poligonos_agricultura.gpkg'

In [42]:
with fiona.open(filename, 
               'w',
               layer='poligonos_agricultura',
               schema = schema,
               crs = crs,
               driver='GPKG') as dst:
    [dst.write(x) for x in fc]

DriverError: sqlite3_exec(CREATE TABLE gpkg_spatial_ref_sys (srs_name TEXT NOT NULL,srs_id INTEGER NOT NULL PRIMARY KEY,organization TEXT NOT NULL,organization_coordsys_id INTEGER NOT NULL,definition  TEXT NOT NULL,description TEXT);INSERT INTO gpkg_spatial_ref_sys (srs_name, srs_id, organization, organization_coordsys_id, definition, description) VALUES ('WGS 84 geodetic', 4326, 'EPSG', 4326, 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]', 'longitude/latitude coordinates in decimal degrees on the WGS 84 spheroid');INSERT INTO gpkg_spatial_ref_sys (srs_name, srs_id, organization, organization_coordsys_id, definition, description) VALUES ('Undefined cartesian SRS', -1, 'NONE', -1, 'undefined', 'undefined cartesian coordinate reference system');INSERT INTO gpkg_spatial_ref_sys (s

# Also even if a geopackage file is copied to LUSTRE it can't be read by ogrinfo

```
cp /home/madmex_user/poligonos_agricultura.gpkg /home/madmex_user/results/copy_geopackage_result/
```

```
ogrinfo -al /home/madmex_user/results/copy_geopackage/poligonos_agricultura.gpkg
ERROR 1: disk I/O error
ERROR 1: disk I/O error
FAILURE:
Unable to open datasource `/home/madmex_user/results/copy_geopackage_result/poligonos_agricultura.gpkg' with the following drivers.
  -> PCIDSK
  -> netCDF
  -> JP2OpenJPEG
  -> PDF
  -> ESRI Shapefile
  -> MapInfo File
  -> UK .NTF
  -> OGR_SDTS
  -> S57
  -> DGN
  -> OGR_VRT
  -> REC
  -> Memory
  -> BNA
  -> CSV
  -> NAS
  -> GML
  -> GPX
  -> LIBKML
  -> KML
  -> GeoJSON
  -> Interlis 1
  -> Interlis 2
  -> OGR_GMT
  -> GPKG
  -> SQLite
  -> OGR_DODS
  -> ODBC
  -> WAsP
  -> PGeo
  -> MSSQLSpatial
  -> OGR_OGDI
  -> PostgreSQL
  -> MySQL
  -> OpenFileGDB
  -> XPlane
  -> DXF
  -> CAD
  -> Geoconcept
  -> GeoRSS
  -> GPSTrackMaker
  -> VFK
  -> PGDUMP
  -> OSM
  -> GPSBabel
  -> SUA
  -> OpenAir
  -> OGR_PDS
  -> WFS
  -> SOSI
  -> HTF
  -> AeronavFAA
  -> Geomedia
  -> EDIGEO
  -> GFT
  -> SVG
  -> CouchDB
  -> Cloudant
  -> Idrisi
  -> ARCGEN
  -> SEGUKOOA
  -> SEGY
  -> XLS
  -> ODS
  -> XLSX
  -> ElasticSearch
  -> Walk
  -> Carto
  -> AmigoCloud
  -> SXF
  -> Selafin
  -> JML
  -> PLSCENES
  -> CSW
  -> VDV
  -> GMLAS
  -> TIGER
  -> AVCBin
  -> AVCE00
  -> HTTP
  
 ```